In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy as sal
from sqlalchemy import create_engine

In [3]:
#Connect to Data Structure for sqlalchemy is: "mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server" --change RPC1 for other volumes

RPC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RPC1.connect()

RC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RC1.connect()


In [4]:
RPC1RibbonInputDF = pd.DataFrame()
RC1RibbonInputDF = pd.DataFrame()

In [5]:
RC1Aiis= ["514", "3679", "2610", "410"]
RPC1Aiis = ["192", "262", "265", "1685", "2710"]

In [6]:
#Pull Data from RPC1
for x in RPC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RPC1)
    RI['Volume']= 'RPC1'
    RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)

RPC1RibbonInputDF

C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\2175985267.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\2175985267.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\2175985267.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\2175985267.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RPC1RibbonInputDF = RPC1RibbonInp

,SourceParent,SourceParentLabel,SourceStructureType,SourceID,TargetID,TargetParent,TargetParentLabel,Notes,PSDArea,Volume
0,48706,BC,73,48707,48651,192,GAC Aii,,8814.989372,RPC1
1,1,CBa,73,24,42960,192,GAC Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,6480.857717,RPC1
2,1,CBa,73,24,42961,192,GAC Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,8332.312104,RPC1
3,1,CBa,73,26244,32501,192,GAC Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,10170.574687,RPC1
4,1,CBa,73,49174,24848,192,GAC Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,15649.935906,RPC1
...,...,...,...,...,...,...,...,...,...,...
66,1069,RodBC,73,30780,38779,2710,GAC Aii,\n\nMay branch here\nX: 12947.4 Y: 10473.2...,36895.515145,RPC1
67,1069,RodBC,73,30783,30791,2710,GAC Aii,\n\nMay branch here\nX: 12947.4 Y: 10473.2...,57889.939095,RPC1
68,1069,RodBC,73,30784,30789,2710,GAC Aii,\n\nMay branch here\nX: 12947.4 Y: 10473.2...,46624.135193,RPC1
69,1069,RodBC,73,25935,38220,2710,GAC Aii,\n\nMay branch here\nX: 12947.4 Y: 10473.2...,50331.894765,RPC1


In [7]:
#Fix any odd names in RPC1 DF or simplify *This is currently a manual step)
df = RPC1RibbonInputDF
df['SourceParentLabel']=df['SourceParentLabel'].str.strip('?()wf3456n-')
df['TargetParentLabel']=df['TargetParentLabel'].str.strip('?()wf3456n-')
df.replace('GAC Aii','Aii', inplace=True)


In [8]:
RPC1RibbonInputDF.SourceParentLabel.value_counts()

RodBC    251
CBa       86
CBb       29
BC         4
Name: SourceParentLabel, dtype: int64

In [9]:
RPC1RibbonInputDF.TargetParent.value_counts()

1685    88
265     81
262     74
2710    71
192     56
Name: TargetParent, dtype: int64

In [10]:
RPC1RibbonInputDF.groupby(["TargetParent"])["SourceParentLabel"].value_counts()

TargetParent  SourceParentLabel
192           RodBC                33
              CBa                  12
              CBb                  10
              BC                    1
262           RodBC                47
              CBa                  16
              CBb                  11
265           RodBC                56
              CBa                  23
              BC                    1
              CBb                   1
1685          RodBC                56
              CBa                  27
              CBb                   5
2710          RodBC                59
              CBa                   8
              BC                    2
              CBb                   2
Name: SourceParentLabel, dtype: int64

In [11]:
RPC1RibbonInputDF = RPC1RibbonInputDF.reset_index(drop = True)

In [12]:
RPC1RibbonInputDF[["SourceParentLabel", "PSDArea"]].groupby("SourceParentLabel").describe()

PSDArea                                           \
                    count          mean           std          min   
SourceParentLabel                                                    
BC                    4.0  14390.888061   9447.733664  4521.590691   
CBa                  86.0  17804.646022  14050.973772  3132.820668   
CBb                  29.0  23900.900314  17751.677395  4489.468865   
RodBC               251.0  38163.057600  25079.052829  2906.364094   

                                                                            
                            25%           50%           75%            max  
SourceParentLabel                                                           
BC                  7741.639702  13882.770979  20532.019338   25276.419598  
CBa                 8736.120361  14177.349347  21751.731581  105502.691336  
CBb                14063.053047  17215.030541  33814.619918   72940.083105  
RodBC              20002.668655  31956.772453  51294.368355  148922.941121

In [13]:
#Pull data from RC1
for x in RC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RC1)
    RI['Volume']= 'RC1'
    RC1RibbonInputDF = RC1RibbonInputDF.append(RI)

RC1RibbonInputDF

C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\567934017.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RC1RibbonInputDF = RC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\567934017.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RC1RibbonInputDF = RC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\567934017.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RC1RibbonInputDF = RC1RibbonInputDF.append(RI)
C:\Users\u0705824\AppData\Local\Temp\ipykernel_23428\567934017.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  RC1RibbonInputDF = RC1RibbonInputDF.append(

,SourceParent,SourceParentLabel,SourceStructureType,SourceID,TargetID,TargetParent,TargetParentLabel,Notes,PSDArea,Volume
0,163230,CBa,73,171178,171179,514,GAC Aii,,32199.363802,RC1
1,65835,CBa,73,160546,160545,514,GAC Aii,,7023.886396,RC1
2,83461,CBa,73,160549,166926,514,GAC Aii,,7948.252316,RC1
3,163236,CBa,73,166955,166956,514,GAC Aii,,56126.856178,RC1
4,166396,CBa,73,166399,166402,514,GAC Aii,,18033.405030,RC1
...,...,...,...,...,...,...,...,...,...,...
97,6561,RodBC,73,17823,17822,410,GAC Aii,,37334.924876,RC1
98,6561,RodBC,73,17824,17825,410,GAC Aii,,16292.710315,RC1
99,6561,RodBC,73,17828,17827,410,GAC Aii,,14562.894360,RC1
100,380,RodBC,73,17878,17877,410,GAC Aii,,32657.695588,RC1


In [14]:
#Fix all expanded names in RC1 (subclasses of BCs beyond a and b are excluded in this analysis)
df = RC1RibbonInputDF
df['SourceParentLabel']=df['SourceParentLabel'].str.strip('?()wf123456n-')
df['TargetParentLabel']=df['TargetParentLabel'].str.strip('?()wf123456n-')
df.replace('GAC Aii','Aii', inplace=True)

In [15]:
RC1RibbonInputDF = RC1RibbonInputDF.reset_index(drop = True)

In [16]:
RC1RibbonInputDF.SourceParentLabel.value_counts()

RodBC    387
CBa       68
CBb        8
BC         1
Name: SourceParentLabel, dtype: int64

In [17]:
RC1RibbonInputDF.TargetParent.value_counts()

514     130
3679    126
2610    106
410     102
Name: TargetParent, dtype: int64

In [18]:
RC1RibbonInputDF.groupby(["TargetParent"])["SourceParentLabel"].value_counts()

TargetParent  SourceParentLabel
410           RodBC                 91
              CBa                   10
              BC                     1
514           RodBC                109
              CBa                   18
              CBb                    3
2610          RodBC                 82
              CBa                   20
              CBb                    4
3679          RodBC                105
              CBa                   20
              CBb                    1
Name: SourceParentLabel, dtype: int64

In [19]:
RC1RibbonInputDF[["SourceParentLabel", "PSDArea"]].groupby("SourceParentLabel").describe()

PSDArea                                            \
                    count          mean           std           min   
SourceParentLabel                                                     
BC                    1.0  45305.922216           NaN  45305.922216   
CBa                  68.0  20394.794197  13118.602544   2659.974295   
CBb                   8.0  24442.003219  18357.271573   3768.755750   
RodBC               387.0  29865.463987  17243.119673   3986.076317   

                                                                            
                            25%           50%           75%            max  
SourceParentLabel                                                           
BC                 45305.922216  45305.922216  45305.922216   45305.922216  
CBa                10640.245500  17812.565652  26983.592090   64113.804668  
CBb                16336.063296  20447.803759  28277.232721   64998.421882  
RodBC              16889.468566  26646.905325  39475.574664  104023.013289

In [20]:
RPC1RibbonInputDF[["SourceParentLabel", "PSDArea"]].groupby("SourceParentLabel").describe()

PSDArea                                           \
                    count          mean           std          min   
SourceParentLabel                                                    
BC                    4.0  14390.888061   9447.733664  4521.590691   
CBa                  86.0  17804.646022  14050.973772  3132.820668   
CBb                  29.0  23900.900314  17751.677395  4489.468865   
RodBC               251.0  38163.057600  25079.052829  2906.364094   

                                                                            
                            25%           50%           75%            max  
SourceParentLabel                                                           
BC                  7741.639702  13882.770979  20532.019338   25276.419598  
CBa                 8736.120361  14177.349347  21751.731581  105502.691336  
CBb                14063.053047  17215.030541  33814.619918   72940.083105  
RodBC              20002.668655  31956.772453  51294.368355  148922.941121

In [21]:
df = RPC1RibbonInputDF
df.loc[df["PSDArea"].idxmax()]

SourceParent                    1243
SourceParentLabel              RodBC
SourceStructureType               73
SourceID                       26011
TargetID                       24871
TargetParent                     192
TargetParentLabel                Aii
Notes                               
PSDArea                148922.941121
Volume                          RPC1
Name: 47, dtype: object

In [22]:
#concatinate the 2 datasets
frames = [RPC1RibbonInputDF, RC1RibbonInputDF]
CombinedRibbonInputDF = pd.concat(frames).reset_index()
CombinedRibbonInputDF

,index,SourceParent,SourceParentLabel,SourceStructureType,SourceID,TargetID,TargetParent,TargetParentLabel,Notes,PSDArea,Volume
0,0,48706,BC,73,48707,48651,192,Aii,,8814.989372,RPC1
1,1,1,CBa,73,24,42960,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,6480.857717,RPC1
2,2,1,CBa,73,24,42961,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,8332.312104,RPC1
3,3,1,CBa,73,26244,32501,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,10170.574687,RPC1
4,4,1,CBa,73,49174,24848,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,15649.935906,RPC1
...,...,...,...,...,...,...,...,...,...,...,...
829,459,6561,RodBC,73,17823,17822,410,Aii,,37334.924876,RC1
830,460,6561,RodBC,73,17824,17825,410,Aii,,16292.710315,RC1
831,461,6561,RodBC,73,17828,17827,410,Aii,,14562.894360,RC1
832,462,380,RodBC,73,17878,17877,410,Aii,,32657.695588,RC1


In [23]:
CombinedRibbonInputDF['PSDArea']= CombinedRibbonInputDF['PSDArea']*1.0e-6
CombinedRibbonInputDF

,index,SourceParent,SourceParentLabel,SourceStructureType,SourceID,TargetID,TargetParent,TargetParentLabel,Notes,PSDArea,Volume
0,0,48706,BC,73,48707,48651,192,Aii,,0.008815,RPC1
1,1,1,CBa,73,24,42960,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.006481,RPC1
2,2,1,CBa,73,24,42961,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.008332,RPC1
3,3,1,CBa,73,26244,32501,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.010171,RPC1
4,4,1,CBa,73,49174,24848,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.015650,RPC1
...,...,...,...,...,...,...,...,...,...,...,...
829,459,6561,RodBC,73,17823,17822,410,Aii,,0.037335,RC1
830,460,6561,RodBC,73,17824,17825,410,Aii,,0.016293,RC1
831,461,6561,RodBC,73,17828,17827,410,Aii,,0.014563,RC1
832,462,380,RodBC,73,17878,17877,410,Aii,,0.032658,RC1


In [24]:
CombinedRibbonInputDF[["SourceParentLabel", "PSDArea", "Volume"]].groupby(["SourceParentLabel", "Volume"]).describe()

PSDArea                                          \
                           count      mean       std       min       25%   
SourceParentLabel Volume                                                   
BC                RC1        1.0  0.045306       NaN  0.045306  0.045306   
                  RPC1       4.0  0.014391  0.009448  0.004522  0.007742   
CBa               RC1       68.0  0.020395  0.013119  0.002660  0.010640   
                  RPC1      86.0  0.017805  0.014051  0.003133  0.008736   
CBb               RC1        8.0  0.024442  0.018357  0.003769  0.016336   
                  RPC1      29.0  0.023901  0.017752  0.004489  0.014063   
RodBC             RC1      387.0  0.029865  0.017243  0.003986  0.016889   
                  RPC1     251.0  0.038163  0.025079  0.002906  0.020003   

                                                        
                               50%       75%       max  
SourceParentLabel Volume                                
BC                RC1     0.045306  0.045306  0.045306  
                  RPC1    0.013883  0.020532  0.025276  
CBa               RC1     0.017813  0.026984  0.064114  
                  RPC1    0.014177  0.021752  0.105503  
CBb               RC1     0.020448  0.028277  0.064998  
                  RPC1    0.017215  0.033815  0.072940  
RodBC             RC1     0.026647  0.039476  0.104023  
                  RPC1    0.031957  0.051294  0.148923

In [25]:
#Use this if/when you want to export something to an excel file (use this for tableau analysis)
CombinedRibbonInputDF.to_excel(r'D:\Documents\Data\RPC1\Aii_GAC_Project\dfExports\CombinedRibbonInput.xlsx')

In [26]:

CombinedRibbonInputDF.groupby("TargetParent")['SourceParent'].value_counts()

TargetParent  SourceParent
192           1001            12
              1243             9
              740              6
              135              5
              26167            5
                              ..
3679          6046             1
              10872            1
              11043            1
              159378           1
              163163           1
Name: SourceParent, Length: 198, dtype: int64

In [27]:
CombinedRibbonInputDF

,index,SourceParent,SourceParentLabel,SourceStructureType,SourceID,TargetID,TargetParent,TargetParentLabel,Notes,PSDArea,Volume
0,0,48706,BC,73,48707,48651,192,Aii,,0.008815,RPC1
1,1,1,CBa,73,24,42960,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.006481,RPC1
2,2,1,CBa,73,24,42961,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.008332,RPC1
3,3,1,CBa,73,26244,32501,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.010171,RPC1
4,4,1,CBa,73,49174,24848,192,Aii,\n*****BEGIN MERGE FROM 26243*****\nPossible b...,0.015650,RPC1
...,...,...,...,...,...,...,...,...,...,...,...
829,459,6561,RodBC,73,17823,17822,410,Aii,,0.037335,RC1
830,460,6561,RodBC,73,17824,17825,410,Aii,,0.016293,RC1
831,461,6561,RodBC,73,17828,17827,410,Aii,,0.014563,RC1
832,462,380,RodBC,73,17878,17877,410,Aii,,0.032658,RC1


In [28]:

# Count the number of unique Source Parents for each Label using value_counts
unique_source_parents = CombinedRibbonInputDF.groupby(['Volume', 'TargetParent', 'SourceParentLabel'])['SourceParent'].nunique().to_frame(name = 'InputCells').reset_index()

# Print the result
print( unique_source_parents)


   Volume  TargetParent SourceParentLabel  InputCells
0     RC1           410                BC           1
1     RC1           410               CBa           6
2     RC1           410             RodBC          12
3     RC1           514               CBa          12
4     RC1           514               CBb           1
5     RC1           514             RodBC          15
6     RC1          2610               CBa          13
7     RC1          2610               CBb           3
8     RC1          2610             RodBC          10
9     RC1          3679               CBa          10
10    RC1          3679               CBb           1
11    RC1          3679             RodBC          14
12   RPC1           192                BC           1
13   RPC1           192               CBa           3
14   RPC1           192               CBb           5
15   RPC1           192             RodBC           6
16   RPC1           262               CBa           7
17   RPC1           262     

In [29]:
SimpRibbonInputDF = CombinedRibbonInputDF.groupby(['Volume', 'TargetParent'])["SourceParentLabel"].value_counts().to_frame(name = 'RibbonCount').reset_index()

In [30]:
df = SimpRibbonInputDF
df["TotalRibbons"] = df.groupby('TargetParent')['RibbonCount'].transform('sum')
df["PercentofInput"] = df["RibbonCount"] / df["TotalRibbons"]*100

print(SimpRibbonInputDF)

   Volume  TargetParent SourceParentLabel  RibbonCount  TotalRibbons  \
0     RC1           410             RodBC           91           102   
1     RC1           410               CBa           10           102   
2     RC1           410                BC            1           102   
3     RC1           514             RodBC          109           130   
4     RC1           514               CBa           18           130   
5     RC1           514               CBb            3           130   
6     RC1          2610             RodBC           82           106   
7     RC1          2610               CBa           20           106   
8     RC1          2610               CBb            4           106   
9     RC1          3679             RodBC          105           126   
10    RC1          3679               CBa           20           126   
11    RC1          3679               CBb            1           126   
12   RPC1           192             RodBC           33          

In [34]:
TotalRPC1Ribbons = SimpRibbonInputDF.loc[SimpRibbonInputDF['Volume'] == 'RPC1', 'RibbonCount'].sum()
TotalRPC1Ribbons

370

In [33]:
TotalRC1Ribbons = SimpRibbonInputDF.loc[SimpRibbonInputDF['Volume'] == 'RC1', 'RibbonCount'].sum()
TotalRC1Ribbons

464

In [35]:
mean_std_df = SimpRibbonInputDF.groupby(['Volume', 'SourceParentLabel'])['PercentofInput'].agg(['mean', 'std'])
mean_std_df

mean       std
Volume SourceParentLabel                     
RC1    BC                  0.980392       NaN
       CBa                14.597754  3.803787
       CBb                 2.291643  1.490032
       RodBC              83.438416  4.848355
RPC1   BC                  1.945728  0.803211
       CBa                22.678936  7.576609
       CBb                 8.491059  7.434606
       RodBC              67.662569  9.356133

In [36]:
Totalmean_std_df = SimpRibbonInputDF.groupby(['Volume'])['TotalRibbons'].agg(['mean', 'std'])
Totalmean_std_df

,mean,std
Volume,,
RC1,116.000000,12.706477
RPC1,73.222222,11.127896


In [ ]:
#Define the CV equation:
cv = lambda x: np.std(x, ddof=1) / np.mean(x) 

SimpRibbonInputDF.groupby(['Volume', 'SourceParentLabel'])['PercentofInput'].apply(cv)

In [37]:
avg_size_df = CombinedRibbonInputDF.groupby(['Volume', 'SourceParentLabel'])['PSDArea'].agg(['mean', 'std'])
avg_size_df

mean       std
Volume SourceParentLabel                    
RC1    BC                 0.045306       NaN
       CBa                0.020395  0.013119
       CBb                0.024442  0.018357
       RodBC              0.029865  0.017243
RPC1   BC                 0.014391  0.009448
       CBa                0.017805  0.014051
       CBb                0.023901  0.017752
       RodBC              0.038163  0.025079

In [ ]:
CombinedRibbonInputDF.query("TargetParent==265")["SourceParent"].value_counts()